In [1]:
from oneibl import one
import numpy as np
import pandas as pd
from brainbox.modeling import glm
from export_funs import trialinfo_to_df
import os

one = one.ONE()

session_id = one.search(subject='ZM_2240', date_range=['2020-01-22', '2020-01-22'],
                        dataset_types=['spikes.clusters'])[0]
spk_t = one.load(session_id, 'spikes.times')[0]
spk_clu = one.load(session_id, 'spikes.clusters')[0]
trialsdf = trialinfo_to_df(session_id, maxlen=2.)
trialsdf = trialsdf.drop(labels=[trialsdf.index[-1], trialsdf.index[0]])


Connected to https://alyx.internationalbrainlab.org as berk.gercek
Connected to https://alyx.internationalbrainlab.org as berk.gercek


In [13]:
import ipywidgets as iw
from brainbox.plot import peri_event_time_histogram
sk1reg = './fits/ZM_2240/2020-01-22_session_2020-06-16_probe0_pyfit_sklearn_1kern_regularized.p'
sk2reg = './fits/ZM_2240/2020-01-22_session_2020-06-16_probe0_pyfit_sklearn_2kern_regularized.p'
sk1noreg = './fits/ZM_2240/2020-01-22_session_2020-06-15_probe0_pyfit_sklearn_1kern_noreg.p'
sk2noreg = './fits/ZM_2240/2020-01-22_session_2020-06-15_probe0_pyfit_sklearn_2kern_noreg.p'
min1 = './fits/ZM_2240/2020-01-22_session_2020-06-15_probe0_pyfit_minimize_1kern.p'
min2 = './fits/ZM_2240/2020-01-22_session_2020-06-15_probe0_pyfit_minimize_2kern.p'
sk1rfit = np.load(sk1reg, allow_pickle=True)
sk2rfit = np.load(sk2reg, allow_pickle=True)
sk1nrfit = np.load(sk1noreg, allow_pickle=True)
sk2nrfit = np.load(sk2noreg, allow_pickle=True)
min1fit = np.load(min1, allow_pickle=True)
min2fit = np.load(min2, allow_pickle=True)
singlekern = {}
twokern = {}
singlekern['reg'] = sk1rfit['weights']
twokern['reg'] = sk2rfit['weights']
singlekern['noreg'] = sk1nrfit['weights']
twokern['noreg'] = sk2nrfit['weights']
singlekern['min'] = min1fit['weights']
twokern['min'] = min2fit['weights']
%matplotlib inline

@iw.interact
def plot_kern(numkerns=[1, 2], cell=singlekern['reg']['stimonL']['weights'].index.to_list(), kern=list(twokern['reg'].keys())):
    if (numkerns == 1) & (kern[-4:] == 'rect'):
        print('No feedback kernel fit on this model')
        return
    if numkerns == 1:
        skregkern = singlekern['reg'][kern]['weights'].loc[cell]
        sknrkern = singlekern['noreg'][kern]['weights'].loc[cell]
        minkern = singlekern['min'][kern]['weights'].loc[cell]
    elif numkerns == 2:
        skregkern = twokern['reg'][kern]['weights'].loc[cell]
        sknrkern = twokern['noreg'][kern]['weights'].loc[cell]
        minkern = twokern['min'][kern]['weights'].loc[cell]
    fig, ax = plt.subplots(2, 2, figsize=(18, 16))
    if kern[:-1] == 'stimon':
        column = 'contrastLeft' if kern[-1] == 'L' else 'contrastRight'
        events = trialsdf[np.isfinite(trialsdf[column])].stimOn_times
        peri_event_time_histogram(spk_t, spk_clu, events, cell, t_before=0, t_after=sk1rfit['kernlen'],
                                  error_bars='sem', ax=ax[0, 0])
    elif kern[-4:] == 'rect':
        key = -1 if kern == 'incorrect' else 1
        events = trialsdf[trialsdf.feedbackType == key].feedback_times
        peri_event_time_histogram(spk_t, spk_clu, events, cell, t_before=0, t_after=sk1rfit['kernlen'],
                                  error_bars='sem', ax=ax[0, 0])
    ax[0, 1].plot(minkern, label='NLL Minimization kernel', c='orange')
    ax[0, 1].set_title('Minimization of NLL')
    ax[1, 0].plot(skregkern, label='Regularized sklearn', c='purple')
    ax[1, 0].set_title('Regularized (alpha = 1) sklearn kernel')
    ax[1, 1].plot(sknrkern, label='Non-regularized sklearn kernel')
    ax[1, 1].set_title('No regularization (alpha = 0) sklearn kernel')
    

interactive(children=(Dropdown(description='numkerns', options=(1, 2), value=1), Dropdown(description='cell', …

In [4]:
singlekern['reg'].keys()

dict_keys(['stimonL', 'stimonR'])

In [4]:
skdf.keys()

dict_keys(['stimonL', 'stimonR'])